In [1]:
# !export JAVA_HOME=~/jdk1.8.0_211
# !export PATH="$JAVA_HOME/bin:$PATH"
import os
os.environ["JAVA_HOME"] = "/home/roberta/jdk1.8.0_211"
os.environ["JRE_HOME"] = "/home/roberta/jdk1.8.0_211/jre"
!which java
os.getenv("JAVA_HOME")

/usr/bin/java


'/home/roberta/jdk1.8.0_211'

In [2]:
import pandas as pd
import numpy as np
import time
import csv
import json
import pyspark as py
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f
import operator
import pagerankPySpark as PRpy
from pyspark.sql.functions import *

Let's open the spark session

In [3]:
spark = SparkSession.builder.appName("SpamPageRank").config("spark.memory.fraction", 0.8).config("spark.executor.memory", "14g").config("spark.driver.memory", "128g").config("spark.sql.shuffle.partitions" , "800").config("spark.driver.maxResultSize", "12g").getOrCreate()

Now we start counting the ougoing and incoming link for the entire graph

In [4]:
df_structure = StructType(fields= [StructField('networkID', StringType(), True), 
                                   StructField('clueWebID', StringType(), True)])
filename = "ClueB-ID-DOCNO.txt"
mapping = spark.read.csv(filename, sep = '\t', schema=df_structure) #148148553 osservazioni

In [5]:
df_structure = StructType(fields= [StructField('GoingFrom', StringType(), True), 
                                   StructField('PointingTo', StringType(), True)])
filename = "ClueWeb09_50m_Network.csv"
networkDF = spark.read.csv(filename, sep = ' ', schema=df_structure) # 

In [ ]:
netw = networkDF.join(mapping, networkDF.GoingFrom == mapping.networkID).select(col('clueWebID').alias('GoingFrom'), 'PointingTo') 
netDF = netw.join(mapping, netw.PointingTo == mapping.networkID).select('GoingFrom',col('clueWebID').alias('PointingTo')) 
netDFpandas= netDF.toPandas()
netDFpandas.to_csv('networkDF.csv', index=False, header = None, sep = " ")

In [4]:
# nodes = mapping.select('clueWebID').toPandas()
# nodes.to_csv('nodes.csv', index=False, header = None)
filename = 'networkDF.csv'
nodesFilename = 'nodes.csv'
number_of_nodes = 148148553

In [6]:
compute_pagerank = False
outgoingLink, incomingLink= PRpy.networkDataComputation(spark, filename, nodesFilename, compute_pagerank, number_of_nodes)#37 iteration

Starting Network Analysis: after this computation we will have for each nodes the Number of Outgoing and Incoming Link and the pagerank score
compute_pagerankFalse
computing incoming link count, outgoing...
Computing the number of Incoming Links
Computing the number of Outgming Links


In [ ]:
incomingLinkDF = incomingLink.toDF()
incomingLink = incomingLinkDF.toPandas()
incomingLink.columns = ["ClueWebID","IncomingLink"]
incomingLink.to_csv("completeIncomingLink.csv", index = False)

In [ ]:
del inncomingLink
del inncomingLinkDF

In [ ]:
outgoingLinkDF = outgoingLink.toDF()
outgoingLink = outgoingLinkDF.toPandas()
outgoingLink.columns = ["ClueWebID","OutgoingLink"]
outgoingLink.to_csv("completeOutgoingLink.csv", index = False)

Now let's compute the same measures for the filtered dataset (the page classified  as spam will not be part of the network

In [ ]:
df_structure = StructType(fields= [StructField('score', IntegerType(), True), 
                                   StructField('clueWebID', StringType(), True)])
filename = "clueweb09spam_Fusion.csv"
spamScore = spark.read.csv(filename, sep = ' ', schema=df_structure) # 503903810 osservazioni di cui 150955774 con uno spam score maggiore di 70

In [ ]:
spamScore = spamScore.join(mapping, on = 'clueWebID')
spamDF = spamScore.filter(spamScore.score>=70) # 61324401
networkDF1 = networkDF.join(spamDF, networkDF.GoingFrom == spamDF.networkID).select(col('clueWebID').alias('GoingFrom'), 'PointingTo') 
#202015555 number of nodes in GoingFrom that are not in SPAM
networkDFwithoutSPAM =  networkDF1.join(spamDF, networkDF1.PointingTo == spamDF.networkID).select('GoingFrom',col('clueWebID').alias('PointingTo')) 
#101776085 number of links in GoingFrom that are not in SPAM

In [ ]:
networkWithoutSPAM = networkDFwithoutSPAM.toPandas()
networkWithoutSPAM.to_csv('networkWithoutSPAM.csv', index=False, header = None, sep = " ")

In [ ]:
alpha = 0.85
max_iter = 100
tol = 1e-6
filename = 'networkWithoutSPAM.csv'
filename = 'networkDF.csv'
nodesFilename = 'nodes.csv'
number_of_nodes = 148148553

In [ ]:
compute_pagerank = False
outgoingLink, incomingLink, score, iteration = PRpy.networkDataComputation(spark, filename, nodesFilename, alpha, max_iter, tol, number_of_nodes)#37 iteration

In [ ]:
rankPySpark = score.toDF()
pandasDF = rankPySpark.toPandas()
pandasDF.columns = ["clueWebID","PR_Score"]
pandasDF.to_csv("SpamPageRankScore.csv", index = False)

In [ ]:
incomingLinkDF = incomingLink.toDF()
incomingLink = incomingLinkDF.toPandas()
incomingLink.columns = ["ClueWebID","IncomingLink"]
incomingLink.to_csv("spamFreeIncomingLink.csv", index = False)

In [ ]:
outgoingLinkDF = outgoingLink.toDF()
outgoingLink = outgoingLinkDF.toPandas()
outgoingLink.columns = ["ClueWebID","OutgoingLink"]
outgoingLink.to_csv("spamFreeOutgoingLink.csv", index = False)